In [12]:
import os

import astropy.table as at
from astropy.constants import G
import astropy.units as u
import h5py
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from tqdm.notebook import tqdm
import thejoker as tj
import tables as tb

from gala.mpl_style import laguna, hesperia

from twobody.transforms import PeKi_to_a

from hq.config import Config
from hq.data import get_rvdata
from hq.physics_helpers import period_at_surface, stellar_radius

In [2]:
figure_path = '../../tex/figures'
os.makedirs(figure_path, exist_ok=True)

In [5]:
c = Config.from_run_name('dr16-random')
alldata, allvisit = c.load_alldata()

In [4]:
gold = at.QTable.read('../../catalogs/gold_sample.fits')
bimodal = at.QTable.read('../../catalogs/bimodal.fits')
bimodal = at.QTable(bimodal[bimodal['LOGG'] > -0.5], masked=False)

INFO: Upgrading Table to masked Table. Use Table.filled() to convert to unmasked table. [astropy.table.table]


In [32]:
low_mass = gold[(gold['m2_min_50'] < 70*u.Mjup) 
                & (gold['m2_min_50'] > 1*u.Mjup)
                & (gold['MAP_s'] < 1.*u.km/u.s)
                & (gold['n_visits'] > 8)
                & (gold['MAP_e'] < 0.6)]
print(len(low_mass))

high_mass = gold[(gold['m2_min_5'] > gold['mass']*u.Msun)
                 & (gold['MAP_s'] < 1.*u.km/u.s)
                 & (gold['n_visits'] > 5)
                 & (gold['MAP_e'] < 0.6)]
print(len(high_mass))

qmin = gold['m2_min_50'].value / gold['mass']
Rstar = stellar_radius(gold['LOGG'], gold['mass']*u.Msun)
a1sini = PeKi_to_a(gold['MAP_P'], gold['MAP_e'], gold['MAP_K'])
asini = a1sini * (1 + 1/qmin)
rlimit = 0.49*qmin**(-2/3.) / (0.6*qmin**(-2/3.) + np.log(1 + qmin**(-1/3)))
roche = gold[(Rstar/asini).decompose() > rlimit]
print(len(roche))

62
19
16


## Low Mass filtering:

In [28]:
# for row in low_mass[np.argsort(low_mass['m2_min_50'])][40:]:
#     apid = row['APOGEE_ID']
#     with h5py.File(c.mcmc_results_path, 'r') as f:
#         samples = tj.JokerSamples.read(f[apid])
#         data = get_rvdata(allvisit[allvisit['APOGEE_ID'] == apid])
        
#         # fig, ax = plt.subplots()
#         # _ = tj.plot_rv_curves(samples[:16], data=data, ax=ax)
                
#         fig, axes = plt.subplots(1, 2, figsize=(9, 4))
        
#         _ = tj.plot_rv_curves(samples[:128], data=data, ax=axes[0])
        
#         sample = samples.median()
#         _ = tj.plot_phase_fold(sample, ax=axes[1], data=data)
#         axes[0].set_title(f"{row['m2_min_50'].to(u.Mjup):.2f}: {apid}",
#                           fontsize=15)
#         axes[1].set_ylabel('')
#         fig.tight_layout()
#         print(apid)

In [30]:
low_mass_fav_ids = [
    '2M04394882-7514363',
    '2M13500861+3249553',
    '2M01083937+8550346',
    '2M17142443-2457219'
]

## High mass filtering

In [37]:
# for row in high_mass[np.argsort(high_mass['m2_min_50'])]:
#     apid = row['APOGEE_ID']
#     with h5py.File(c.mcmc_results_path, 'r') as f:
#         samples = tj.JokerSamples.read(f[apid])
#         data = get_rvdata(allvisit[allvisit['APOGEE_ID'] == apid])
                
#         fig, axes = plt.subplots(1, 2, figsize=(9, 4))
        
#         _ = tj.plot_rv_curves(samples[:128], data=data, ax=axes[0])
        
#         sample = samples.median()
#         _ = tj.plot_phase_fold(sample, ax=axes[1], data=data)
#         axes[0].set_title(f"{row['m2_min_50'].to(u.Msun):.2f}: {apid}",
#                           fontsize=15)
#         axes[1].set_ylabel('')
#         fig.tight_layout()
#         print(apid)

In [38]:
high_mass_fav_ids = [
    '2M13090983+1711572',
    '2M14332948+0958420',
    '2M13143277+1702017',
    '2M00574795+8502290'
]